# Real-world network

In [1]:
import networkx as nx
from networkx.algorithms.centrality import *
import numpy as np
import operator
from analysis import *

In [2]:
g = nx.read_gml("./RealNetworks/twitter_copenhagen.gml")

In [3]:
print(nx.info(g))

Graph with 761 nodes and 1029 edges


<ipython-input-3-99dac2172e06>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))


# Check connectedness

In [4]:
nx.is_connected(g)

True

In [5]:
nx.number_connected_components(g)

1

# Get LCC if graph is disconnected

# Relabel it all if nodes are strings
- To be able to run all types of correlation together
- But put the graph relabled in `g_relabled`

In [6]:
from myTruss import mappingAndRelabeling

In [7]:
g_relabled = mappingAndRelabeling(g)

In [8]:
print(nx.info(g))
print(nx.info(g_relabled))

Graph with 761 nodes and 1029 edges
Graph with 761 nodes and 1029 edges


<ipython-input-8-632c4a2d19f6>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))
<ipython-input-8-632c4a2d19f6>:2: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g_relabled))


In [9]:
g = g_relabled

# Infer community structure with Louvain

In [10]:
nx.__version__

'2.8.4'

In [11]:
import pandas as pd
from os.path  import exists
import infomap as im

In [12]:
im.__version__

'2.3.0'

In [13]:
G = g

In [14]:
partition   = nx.algorithms.community.louvain_communities(G, seed = 0)
partition_Q = nx.algorithms.community.modularity(G, partition)

for seed in Bar("Progress", check_tty = False).iter(range(1,5)):
    partition_   = nx.algorithms.community.louvain_communities(G, seed = seed)
    partition_Q_ = nx.algorithms.community.modularity(G, partition_)
    
    if partition_Q_ > partition_Q:
        partition   = partition_
        partition_Q = partition_Q_

Progress |################################| 4/4


In [15]:
communities = { community_ID : set(community) for (community_ID, community) in enumerate(partition, start = 1)}

In [16]:
community_assignments = dict()
for (community_ID, community) in communities.items():
    for node in community:
        community_assignments[node] = community_ID

In [17]:
results_file = "./results/twitter_copenhagen-modularity.csv"

if not exists(results_file):
    # Map Equation Centrality
    tree_file = "./results/twitter_copenhagen-temp.tree"

    # use Infomap to write a tree file that we change to reflect the Louvain partition
    infomap = im.Infomap(silent = True, num_trials = 1, directed = False, two_level = True, seed = 42)
    infomap.add_networkx_graph(G)
    infomap.run()
    infomap.write_tree(tree_file)
    
    infomap_tree = dict()

    # read the tree file
    with open(tree_file, "r") as fh:
        for line in fh:
            if not line.startswith("#"):
                _, flow, name, node_id = line.strip().split(" ")
                name = name[1:-1]
                infomap_tree[int(name)] = (flow, node_id)
    
    # write the Louvain partition to an Infomap tree file
    with open("./results/twitter_copenhagen-modularity.tree", "w") as fh:
        for community_ID, community in enumerate(partition, start = 1):
            for node_pos, node in enumerate(community, start = 1):
                path = f"{community_ID}:{node_pos}"
                flow, node_id = infomap_tree[node]
                fh.write(f"{path} {flow} \"{node}\" {node_id}\n")
    
    # Load the Louvain partition with Infomap and use unrecorded link teleportation
    infomap2 = im.Infomap("--silent --no-infomap --cluster-data ./results/twitter_copenhagen-modularity.tree")
    node_mapping = infomap2.add_networkx_graph(G)
    infomap2.run()
    
    mec_ranks = [node for node, _score in sorted( [ (node_mapping[node.node_id], node.modular_centrality) for node in infomap2.iterLeafNodes() ]
                                                 , key     = second
                                                 , reverse = True
                                                 )]

    
    index = []
    for node in G.nodes:
        index.append(node)

    results = pd.DataFrame(index=index)

    results["map equation centrality rank"]       = toRanking(mec_ranks,        results.index)
    results.to_csv(results_file)

else:
    results = pd.read_csv(results_file, index_col = 0)

    mec_ranks         = mkRanks(results["map equation centrality rank"])

In [18]:
len(mec_ranks)

761

In [19]:
# node 107 has rank 0 (i.e. 1)
# node 1684 has rank 1 (i.e. 2)
# ...
# node 0 has rank 4 (i.e. 5)
mec_ranks

[137,
 105,
 16,
 57,
 70,
 28,
 9,
 18,
 184,
 131,
 264,
 115,
 291,
 33,
 164,
 176,
 2,
 25,
 19,
 260,
 154,
 23,
 50,
 48,
 282,
 6,
 27,
 45,
 0,
 171,
 142,
 211,
 355,
 151,
 221,
 147,
 554,
 224,
 369,
 361,
 97,
 75,
 55,
 22,
 130,
 216,
 213,
 215,
 262,
 179,
 385,
 183,
 187,
 53,
 60,
 44,
 193,
 79,
 51,
 242,
 375,
 411,
 315,
 515,
 166,
 14,
 90,
 360,
 125,
 255,
 297,
 112,
 329,
 209,
 285,
 186,
 476,
 338,
 328,
 581,
 58,
 230,
 127,
 634,
 159,
 197,
 589,
 64,
 563,
 350,
 86,
 157,
 126,
 305,
 382,
 436,
 185,
 217,
 218,
 435,
 595,
 207,
 41,
 455,
 319,
 199,
 512,
 677,
 146,
 225,
 222,
 394,
 236,
 658,
 276,
 47,
 169,
 168,
 271,
 272,
 161,
 258,
 458,
 15,
 32,
 440,
 122,
 306,
 383,
 299,
 140,
 181,
 590,
 433,
 109,
 278,
 5,
 397,
 3,
 212,
 220,
 323,
 120,
 342,
 119,
 38,
 324,
 61,
 504,
 101,
 49,
 117,
 102,
 145,
 195,
 513,
 701,
 150,
 525,
 36,
 69,
 368,
 628,
 399,
 565,
 501,
 502,
 289,
 469,
 24,
 461,
 172,
 152,
 191,
 82,


In [20]:
counter = len(mec_ranks)
dict_map_eq_centrality = {}

In [21]:
for node in mec_ranks:
    dict_map_eq_centrality[int(node)] = counter
    counter = counter - 1

In [22]:
dict_map_eq_centrality

{137: 761,
 105: 760,
 16: 759,
 57: 758,
 70: 757,
 28: 756,
 9: 755,
 18: 754,
 184: 753,
 131: 752,
 264: 751,
 115: 750,
 291: 749,
 33: 748,
 164: 747,
 176: 746,
 2: 745,
 25: 744,
 19: 743,
 260: 742,
 154: 741,
 23: 740,
 50: 739,
 48: 738,
 282: 737,
 6: 736,
 27: 735,
 45: 734,
 0: 733,
 171: 732,
 142: 731,
 211: 730,
 355: 729,
 151: 728,
 221: 727,
 147: 726,
 554: 725,
 224: 724,
 369: 723,
 361: 722,
 97: 721,
 75: 720,
 55: 719,
 22: 718,
 130: 717,
 216: 716,
 213: 715,
 215: 714,
 262: 713,
 179: 712,
 385: 711,
 183: 710,
 187: 709,
 53: 708,
 60: 707,
 44: 706,
 193: 705,
 79: 704,
 51: 703,
 242: 702,
 375: 701,
 411: 700,
 315: 699,
 515: 698,
 166: 697,
 14: 696,
 90: 695,
 360: 694,
 125: 693,
 255: 692,
 297: 691,
 112: 690,
 329: 689,
 209: 688,
 285: 687,
 186: 686,
 476: 685,
 338: 684,
 328: 683,
 581: 682,
 58: 681,
 230: 680,
 127: 679,
 634: 678,
 159: 677,
 197: 676,
 589: 675,
 64: 674,
 563: 673,
 350: 672,
 86: 671,
 157: 670,
 126: 669,
 305: 668,
 

In [23]:
# Write Centrality
with open('./MapEquationCentralityResults/OSN-Twitter/dict_map_eq_centrality.txt', 'w') as f:
    for key, value in dict_map_eq_centrality.items():
        f.write('%s:%s\n' % (key, value))

In [24]:
# Read Centrality
dict_map_eq_centrality_read = dict()
with open('./MapEquationCentralityResults/OSN-Twitter/dict_map_eq_centrality.txt') as raw_data:
    for item in raw_data:
        if ':' in item:
            key,value = item.split(':', 1)
            value = value.replace('\n', '')
            dict_map_eq_centrality_read[int(key)]=float(value)
        else:
            pass # deal with bad lines of text here

In [25]:
dict_map_eq_centrality_read == dict_map_eq_centrality

True